# Вебинар 4. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

# Домашнее задание

1. Перенесите метрики в модуль src.metrics.py
2. Перенесите функцию prefilter_items в модуль src.utils.py
3. Создайте модуль src.recommenders.py. Напищите код для класса ниже 
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в src.recommenders.py
4. Проверьте, что все модули корректно импортируются

In [1]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
# from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight



Проверка, что все работает

In [2]:
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path) 

from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender



In [3]:
data = pd.read_csv('../transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.tail(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
2488949,98,41944918665,684,9526886,2,0.6,421,0.0,1705,98,0.0,0.0
2488950,98,41944918665,684,15972479,1,1.0,421,0.0,1705,98,0.0,0.0


In [4]:
data_train.shape

(2485538, 12)

In [5]:
item_features = pd.read_csv('../product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [6]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [7]:
# checking number of items before and after filtering
n_items_before = data_train['item_id'].nunique()

data_train_prefiltered = prefilter_items(data_train)

n_items_after = data_train_prefiltered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 90386 to 5000


In [8]:
data_train_prefiltered.shape

(1576221, 12)

In [9]:
# we may use more prefiltering on price and departments

In [10]:
my_data = MainRecommender(data_train_prefiltered)

In [11]:
my_data.get_similar_items_recommendation(98, num=5)

[1034686, 1030040, 884945, 1080110, 929309]